In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [24]:
import pandas as pd
import numpy as np
import json,ast
from scipy.sparse import csr_matrix as csr
# SKLEARN
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, pairwise_distances
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors

In [3]:
movies = pd.read_csv('/content/drive/My Drive/movies_dataset.zip (Unzipped Files)/movies_data.csv')
print(movies.head(10))
ratings = pd.read_csv('/content/drive/My Drive/movies_dataset.zip (Unzipped Files)/ratings.csv')
print(ratings.head(10))

                                              genres  ...                                              title
0  [{'id': 14, 'name': 'Fantasy'}, {'id': 35, 'na...  ...                                           Reckless
1  [{'id': 10749, 'name': 'Romance'}, {'id': 12, ...  ...                               Girl in the Cadillac
2                [{'id': 99, 'name': 'Documentary'}]  ...           The Haunted World of Edward D. Wood, Jr.
3                [{'id': 99, 'name': 'Documentary'}]  ...             Carmen Miranda: Bananas Is My Business
4                      [{'id': 18, 'name': 'Drama'}]  ...                           Other Voices Other Rooms
5  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...  ...                              Dancer, Texas Pop. 81
6                      [{'id': 18, 'name': 'Drama'}]  ...                                  The Farmer's Wife
7                     [{'id': 35, 'name': 'Comedy'}]  ...                                   Stiff Upper Lips
8  [{'id': 18, 'nam

Preprocessing

In [4]:
print(movies.shape,"\n")
movies.info()

(45466, 4) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genres    45466 non-null  object
 1   id        45466 non-null  object
 2   overview  44512 non-null  object
 3   title     45460 non-null  object
dtypes: object(4)
memory usage: 1.4+ MB


In [5]:
# movies['movieId'] = [int(eval(s)[0]['id']) if len(eval(s))>0 else 0 for s in movies['genres'].values]
# Extracting the genres
def extractGenre(s):
  lst = eval(s)
  if len(lst) > 0:
    return lst[0]['name']
  else:
    return ''
# Removing NaN
def delNan(cell):
  if type(cell) == float:
    return ''
  else:
    return cell
# Test run
'''print(extractGenre([{'id': 14, 'name': 'Fantasy'}, {'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}]))'''

movies['genres'] = movies['genres'].apply(extractGenre)
movies['title'] = movies['title'].apply(delNan)
movies['overview'] = movies['overview'].apply(delNan)
movies

,genres,id,overview,title
0,Fantasy,58372,"On Christmas eve, a relentlessly cheerful woma...",Reckless
1,Romance,278939,A runaway teenage girl and a drifter rob a ban...,Girl in the Cadillac
2,Documentary,73183,The strange life and the wonderfully awful fil...,"The Haunted World of Edward D. Wood, Jr."
3,Documentary,255546,A biography of the Portuguese-Brazilian singer...,Carmen Miranda: Bananas Is My Business
4,Drama,64562,Truman Capote's semi-autobiographical first no...,Other Voices Other Rooms
...,...,...,...,...
45461,Carousel Productions,20-08-1997,Released,
45462,Animation,122662,Third film of the Mardock Scramble series.,
45463,Aniplex,29-09-2012,Released,
45464,TV Movie,249260,A group of skiers are terrorized during spring...,


In [6]:
# Movies without titles have no relevance in recommendation systems. So, since the number of movies with missing title are only 6 out of the 45466 movies, we remove them
movies.drop(index=[45460,45461,45462,45463,45464,45465],inplace=True)
movies['id'] = movies['id'].astype(int)
# Merging genres and overview together so that our recommender system is based on both the overview and genre
movies['genre+overview'] = [",".join([movies.iloc[i,0].lower(),movies.iloc[i,2].lower()]) for i in range(movies.shape[0])]
# Counting the number of movies under each genre
print(movies['genres'].value_counts(),"\n")
mov_enc = movies.copy()
# Label Encoding the genres
le = LabelEncoder()
mov_enc['genres'] = le.fit_transform(mov_enc['genres'])
print("ORIGINAL :\n",movies,"\n\nENCODED GENRES :\n",mov_enc)

Drama              11966
Comedy              8820
Action              4488
Documentary         3415
Horror              2619
                    2442
Crime               1685
Thriller            1665
Adventure           1514
Romance             1191
Animation           1123
Fantasy              704
Science Fiction      647
Mystery              554
Family               524
Music                487
Western              451
TV Movie             389
War                  379
History              279
Foreign              118
Name: genres, dtype: int64 

ORIGINAL :
             genres  ...                                     genre+overview
0          Fantasy  ...  fantasy,on christmas eve, a relentlessly cheer...
1          Romance  ...  romance,a runaway teenage girl and a drifter r...
2      Documentary  ...  documentary,the strange life and the wonderful...
3      Documentary  ...  documentary,a biography of the portuguese-braz...
4            Drama  ...  drama,truman capote's semi-autobio

Training the model for content based recommendation

In [7]:
# Training the tf-idf model on the genre+overview column for a subset of the data
movies_new = movies.iloc[:35001,:]
tfidf = TfidfVectorizer(analyzer='word',ngram_range=(1,3),stop_words='english',lowercase=True,encoding='utf-8')
tfidf_matrix = tfidf.fit_transform(movies_new['genre+overview'])
sim_scores = linear_kernel(tfidf_matrix,tfidf_matrix)

In [8]:
# Function for recommending movies
indices = pd.Series(movies_new.index,index=movies_new['id']).drop_duplicates()
def rec_movie(movieId,similarity=sim_scores):
  # User input is mapped to the corresponding movie id
  index = indices[movieId]
  # Pairwise score evaluation
  similar = list(enumerate(similarity[index]))
  # Sort in decreasing order
  similar = sorted(similar, key = lambda x: x[1], reverse=True)
  # Fetch the top 10 movies
  mov10_ind = [i[0] for i in similar[1:11]]
  recommended = pd.DataFrame()
  recommended['movieId'],recommended['title'],recommended['genre+overview'] = movies_new['id'].iloc[mov10_ind],movies_new['title'].iloc[mov10_ind],movies_new['genre+overview'].iloc[mov10_ind]
  return recommended

# Test for Despicable Me
print("ORIGINAL MOVIE : ", movies_new['title'].iloc[indices[20352]],"\nGENRE + OVERVIEW : ", movies_new['genre+overview'].iloc[indices[20352]])
print("\n\n10 RECOMMENDED MOVIES :\n")
rec_movie(20352)

ORIGINAL MOVIE :  Despicable Me 
GENRE + OVERVIEW :  animation,villainous gru lives up to his reputation as a despicable, deplorable and downright unlikable guy when he hatches a plan to steal the moon from the sky. but he has a tough time staying on task after three orphans land in his care.


10 RECOMMENDED MOVIES :



,movieId,title,genre+overview
20438,288710,The Frontier,"crime,a desperate young woman, on the run from..."
389,448879,Приключения Домовёнка,"animation,"
460,84774,Alice in Wonderland,"animation,"
15768,221171,Kilplased,"animation,"
19201,213321,Investigation Held by Kolobki,"animation,"
29559,343693,Aladin,"animation,"
29784,44486,Mars,"animation,"
12190,46059,The Hot Rock,"crime,dortmunder and his pals plan to steal a ..."
34547,93556,Prairie Love,"drama,as a mysterious high plains drifter trav..."
19302,15171,Charlotte's Web,"animation,wilbur the pig is scared of the end ..."


Predicting ratings

In [9]:
print(ratings.shape,"\n")
ratings.info()

(100004, 3) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   userId   100004 non-null  int64  
 1   movieId  100004 non-null  int64  
 2   rating   100004 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 2.3 MB


In [10]:
# Using a subset of the data due to RAM limitations
ratings_new = ratings.iloc[:20001,:]
# Splitting to train and test data
train,test = ratings_new.iloc[:(int(0.8*ratings_new.shape[0])+1),:],ratings_new.iloc[(int(0.8*ratings_new.shape[0])+1):,:]
print("TRAIN :\n",train,"\n\nTEST :\n",test)

TRAIN :
        userId  movieId  rating
0           2       17     5.0
1           2       39     5.0
2           2      150     5.0
3           2      222     5.0
4           2      265     5.0
...       ...      ...     ...
15996      77     3070     4.5
15997      77     3275     4.5
15998      77     3421     4.5
15999      77     3994     4.5
16000      77     4011     4.5

[16001 rows x 3 columns] 

TEST :
        userId  movieId  rating
16001      77     4499     4.5
16002      77     6755     4.5
16003      77     7099     4.5
16004      77     7387     4.5
16005      77     7649     4.5
...       ...      ...     ...
19996     442      597     4.5
19997     442      858     4.5
19998     442     1035     4.5
19999     442     1036     4.5
20000     442     1049     4.5

[4000 rows x 3 columns]


In [19]:
userID,movieId = 2,222
test1 = rec_movie(movieId)
test1

,movieId,title,genre+overview
219,8930,Modern Life,"documentary,"
632,130638,Hitler - Never Heard of Him,"documentary,"
1439,2692,The Red Elvis,"documentary,"
1660,359807,Chico - Artista Brasileiro,"documentary,"
1842,449863,Cantábrico: Los dominios del oso pardo,"documentary,"
3964,32547,Social Genocide,"documentary,"
4892,292539,Food Chains,"documentary,"
4929,150049,The New Watchdogs,"documentary,"
8569,62130,El Big Bang de Tunguska,"documentary,"
9468,408550,Les Pépites,"documentary,"


In [21]:
final1 = ratings.merge(test1,on='movieId')
final1

,userId,movieId,rating,title,genre+overview
0,23,2692,5.0,The Red Elvis,"documentary,"
1,75,2692,5.0,The Red Elvis,"documentary,"
2,99,2692,5.0,The Red Elvis,"documentary,"
3,111,2692,5.0,The Red Elvis,"documentary,"
4,118,2692,5.0,The Red Elvis,"documentary,"
...,...,...,...,...,...
77,94,2692,2.5,The Red Elvis,"documentary,"
78,265,2692,2.0,The Red Elvis,"documentary,"
79,133,2692,0.5,The Red Elvis,"documentary,"
80,481,8930,4.5,Modern Life,"documentary,"


In [23]:
pred = final1['rating'].median()
pred

4.5

In [16]:
test2 = ratings.merge(movies_new,left_on='movieId',right_on='id')
test2

,userId,movieId,rating,genres,id,overview,title,genre+overview
0,2,17,5.0,Horror,17,Adèle and her daughter Sarah are traveling on ...,The Dark,"horror,adèle and her daughter sarah are travel..."
1,57,17,5.0,Horror,17,Adèle and her daughter Sarah are traveling on ...,The Dark,"horror,adèle and her daughter sarah are travel..."
2,86,17,5.0,Horror,17,Adèle and her daughter Sarah are traveling on ...,The Dark,"horror,adèle and her daughter sarah are travel..."
3,121,17,5.0,Horror,17,Adèle and her daughter Sarah are traveling on ...,The Dark,"horror,adèle and her daughter sarah are travel..."
4,161,17,5.0,Horror,17,Adèle and her daughter Sarah are traveling on ...,The Dark,"horror,adèle and her daughter sarah are travel..."
...,...,...,...,...,...,...,...,...
42829,581,2191,0.5,Thriller,2191,A team of astronauts on the first mission to M...,Stranded,"thriller,a team of astronauts on the first mis..."
42830,581,5413,0.5,Drama,5413,The movie starts at the 1998 bomb attack by th...,Omagh,"drama,the movie starts at the 1998 bomb attack..."
42831,581,54768,0.5,Comedy,54768,"A new kid moves into school, making enemies wi...",Zapped Again!,"comedy,a new kid moves into school, making ene..."
42832,624,3933,0.5,Romance,3933,"Set in a 19th-century european village, this s...",Corpse Bride,"romance,set in a 19th-century european village..."


In [15]:
temp = train.merge(movies_new,left_on='movieId',right_on='id')
temp

,userId,movieId,rating,genres,id,overview,title,genre+overview
0,2,17,5.0,Horror,17,Adèle and her daughter Sarah are traveling on ...,The Dark,"horror,adèle and her daughter sarah are travel..."
1,57,17,5.0,Horror,17,Adèle and her daughter Sarah are traveling on ...,The Dark,"horror,adèle and her daughter sarah are travel..."
2,86,17,5.0,Horror,17,Adèle and her daughter Sarah are traveling on ...,The Dark,"horror,adèle and her daughter sarah are travel..."
3,121,17,5.0,Horror,17,Adèle and her daughter Sarah are traveling on ...,The Dark,"horror,adèle and her daughter sarah are travel..."
4,161,17,5.0,Horror,17,Adèle and her daughter Sarah are traveling on ...,The Dark,"horror,adèle and her daughter sarah are travel..."
...,...,...,...,...,...,...,...,...
7342,73,59995,4.5,Drama,59995,A lyrical telling of the coming of age of a 13...,Wild Tigers I Have Known,"drama,a lyrical telling of the coming of age o..."
7343,73,72733,4.5,Comedy,72733,Mei Li is a 30 year old woman who has one bad ...,Bangkok Traffic Love Story,"comedy,mei li is a 30 year old woman who has o..."
7344,73,110501,4.5,Comedy,110501,Sedam I Po is a Serbian movie. Seven stories f...,Seven and a Half,"comedy,sedam i po is a serbian movie. seven st..."
7345,76,44694,4.5,Documentary,44694,Bringing his unique sense of humor to this biz...,Big Time,"documentary,bringing his unique sense of humor..."


In [47]:
pvt = ratings_new.pivot_table(index='userId',columns='movieId',values='rating')
pvt.fillna(0,inplace=True)
pvt

movieId,1,2,3,5,6,7,8,9,10,11,12,13,14,16,17,18,19,20,21,22,23,24,25,26,28,29,30,31,32,34,35,36,39,40,41,43,45,46,47,48,...,140741,140743,140745,140747,140749,140751,140755,140759,140761,140763,141124,141668,141886,142068,142240,142488,142997,143385,144976,145935,146656,147037,148372,148626,148881,149406,150548,150856,152017,152077,152081,153584,156025,157296,160438,160590,161944,162376,162542,163949
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
cosine_sim = 1-pairwise_distances(pvt, metric="cosine")
cosine_sim

array([[1.        , 0.        , 0.0276395 , ..., 0.        , 0.        ,
        0.13834289],
       [0.        , 1.        , 0.09261065, ..., 0.17418325, 0.10503645,
        0.15984152],
       [0.0276395 , 0.09261065, 1.        , ..., 0.09166985, 0.        ,
        0.14721354],
       ...,
       [0.        , 0.17418325, 0.09166985, ..., 1.        , 0.        ,
        0.11470787],
       [0.        , 0.10503645, 0.        , ..., 0.        , 1.        ,
        0.06917145],
       [0.13834289, 0.15984152, 0.14721354, ..., 0.11470787, 0.06917145,
        1.        ]])

In [49]:
pearson_sim = 1-pairwise_distances(pvt, metric="correlation")
pearson_sim

array([[ 1.        , -0.00267612,  0.01824194, ..., -0.00178529,
        -0.00296336,  0.13500379],
       [-0.00267612,  1.        ,  0.08530793, ...,  0.17287512,
         0.10264289,  0.15692357],
       [ 0.01824194,  0.08530793,  1.        , ...,  0.08726173,
        -0.00989176,  0.13699313],
       ...,
       [-0.00178529,  0.17287512,  0.08726173, ...,  1.        ,
        -0.00178529,  0.11271466],
       [-0.00296336,  0.10264289, -0.00989176, ..., -0.00178529,
         1.        ,  0.06555248],
       [ 0.13500379,  0.15692357,  0.13699313, ...,  0.11271466,
         0.06555248,  1.        ]])

In [73]:
# Get 10 similar users by nearest neighbors algorithm, defaulting to Pearson correlation coefficient metric
def sim10_users(user_id,pvt,metric="correlation",k=10):
  indices_sim = []
  knn = NearestNeighbors(metric = metric, algorithm = 'brute') 
  knn.fit(pvt)
  distances, indices_sim = knn.kneighbors(pvt.iloc[user_id - 1, :].values.reshape(1, -1), n_neighbors = k+1)
  sims = 1 - distances.flatten()
  return sims,indices_sim

# Predict ratings
'''ALGORITHM :
1. Get 10 simialr users
2. Get the mean of all user ratings for that userId
3. Find sum of all ratings of the similar users obtained.
4. for all similar users:
    --> Find rating_diff = (Rating by user j on movie i) - (mean of all user ratings by that user)
    --> Get updated_rating = updated_rating + (rating_diff * similarity_score(of user j))
5. Predicted rating = (((step 2.)*updated_rating + step 3.) + 1)
'''
def predict_rating(user_id, movie_id, pvt, metric = "correlation", k = 10):
    pred = 0
    indices_mov = list(pvt.columns)
    indexm = indices_mov.index(movie_id)
    # STEP 1
    sims, indices = sim10_users(user_id, pvt, metric, k)
    # STEP 2
    mean_rating = pvt.loc[user_id,:].mean()     # Adjusting for zero based indexing
    # STEP 3
    rtSum = np.sum(sims) - 1
    pdt,updated_rating = 1,0           # Initializing product and updated rating
    # STEP 4
    for i in range(0, len(indices.flatten())):
        if (indices.flatten()[i] + 1) == user_id:
            continue
        else: 
            rating_diff = abs(pvt.iloc[indices.flatten()[i],indexm]-np.mean(pvt.iloc[indices.flatten()[i],:]))
            pdt = rating_diff * (sims[i])
            updated_rating = updated_rating + pdt
    
    pred = pred + ((mean_rating*updated_rating + rtSum) + 1)
    return pred
# Test
predict_rating(77, 4499, pvt)

3.103773280334006

Final test runs for :

1. Berlin : Symphony of a Great City
2. Pepi, Luci, Bom

In [78]:
def predictRating_combined(userId,movieId,pvt=pvt,metric="correlation",k=10):
  testdf1 = rec_movie(movieId)
  finaldf1 = ratings.merge(testdf1,on='movieId')
  if len(list(finaldf1.index)) > 0 :
    pred = final1['rating'].median()
    return pred,testdf1
  else:
    pred2 = predict_rating(userId,movieId,pvt=pvt,metric="correlation",k=10)
    return pred2,testdf1

# Final function for predicting user rating and recommended movies
def recommend(userId,movieId,movies_new = movies_new):
  indices = pd.Series(movies_new.index,index=movies_new['id']).drop_duplicates()
  predicted_rating,rec = predictRating_combined(userId,movieId)
  print('\nRECOMMENDED MOVIES for the movie {} : \n\n{}\n\n=== Predicted rating for user {} -> movie {}: {:.1f} ==='.format(movies_new['title'].iloc[indices[movieId]],rec,userId,movieId,predicted_rating))

# Test run 1
recommend(2,222)
print('-'*90)
# Test run 2
recommend(77,4499)


RECOMMENDED MOVIES for the movie Berlin: Symphony of a Great City : 

      movieId                                   title genre+overview
219      8930                             Modern Life   documentary,
632    130638             Hitler - Never Heard of Him   documentary,
1439     2692                           The Red Elvis   documentary,
1660   359807              Chico - Artista Brasileiro   documentary,
1842   449863  Cantábrico: Los dominios del oso pardo   documentary,
3964    32547                         Social Genocide   documentary,
4892   292539                             Food Chains   documentary,
4929   150049                       The New Watchdogs   documentary,
8569    62130                 El Big Bang de Tunguska   documentary,
9468   408550                             Les Pépites   documentary,

=== Predicted rating for user 2 -> movie 222: 4.5 ===
------------------------------------------------------------------------------------------

RECOMMENDED MOVIES for 